# Word List Post-Processing & Cleanup

This notebook cleans up `valid_words.csv` by:
- **Removing trailing apostrophes** (word' → removed)
- **Filtering 2-letter lowercase words** (except 'a' and 'i')
- **Detecting and fixing 500+ acronyms** (Ny → NY, Fbi → FBI, Covid → COVID)

**Input:** `valid_words.csv` (word, count)

**Output:** `cleaned_words.csv` (cleaned and properly formatted)

## Step 1: Install Dependencies

In [ ]:
# Install required libraries
!pip install pandas

## Step 2: Upload Your CSV File

**Instructions:**
1. Click the **Folder icon** on the left sidebar
2. Click the **Upload** button
3. Select `valid_words.csv` from your computer
4. Wait for the upload to complete

## Step 3: Import Libraries and Define Comprehensive Acronym List

In [ ]:
import pandas as pd
import re

# COMPREHENSIVE ACRONYM LIST (500+ acronyms)
# All acronyms that should be fully uppercase
KNOWN_ACRONYMS = {
    # === US STATES (50) ===
    'al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'fl', 'ga',
    'hi', 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md',
    'ma', 'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj',
    'nm', 'ny', 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc',
    'sd', 'tn', 'tx', 'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy',
    
    # === MAJOR CITIES ===
    'nyc', 'la', 'sf', 'dc', 'philly',
    
    # === COUNTRIES & REGIONS ===
    'usa', 'uk', 'eu', 'uae', 'ussr', 'prc', 'drc', 'uae', 'gcc',
    
    # === GOVERNMENT & AGENCIES ===
    'fbi', 'cia', 'nsa', 'dea', 'atf', 'dhs', 'tsa', 'ice', 'cbp',
    'epa', 'fda', 'fcc', 'ftc', 'sec', 'irs', 'ssa', 'osha',
    'dod', 'doj', 'hhs', 'hud', 'dot', 'doe', 'va', 'dhs',
    'nasa', 'noaa', 'usda', 'usps', 'dmv', 'cdc', 'nih',
    'fema', 'darpa', 'nist', 'usgs', 'nrc', 'nlrb',
    
    # === MILITARY ===
    'usaf', 'usmc', 'usn', 'army', 'navy', 'nato', 'un', 'seals',
    'raf', 'sas', 'kgb', 'mi5', 'mi6', 'mossad', 'cia', 'nsa',
    
    # === HEALTH & MEDICAL ===
    'hiv', 'aids', 'covid', 'sars', 'mers', 'h1n1', 'ebola',
    'dna', 'rna', 'mrna', 'cpr', 'icu', 'er', 'emt', 'ems',
    'adhd', 'ptsd', 'ocd', 'bmi', 'cdc', 'who', 'nih',
    'std', 'sti', 'hpv', 'tb', 'ms', 'als', 'md', 'rn', 'lpn',
    
    # === TECHNOLOGY - GENERAL ===
    'it', 'ai', 'ml', 'nlp', 'ocr', 'iot', 'ar', 'vr', 'xr',
    'api', 'sdk', 'ide', 'gui', 'cli', 'ui', 'ux', 'qa', 'qc',
    'cpu', 'gpu', 'ram', 'rom', 'ssd', 'hdd', 'usb', 'hdmi',
    'wifi', 'lan', 'wan', 'vpn', 'ip', 'tcp', 'udp', 'dns',
    'dhcp', 'nat', 'ssh', 'ssl', 'tls', 'https', 'http', 'ftp',
    'smtp', 'pop', 'imap', 'voip', 'sip', 'rtp', 'rtsp',
    'cdn', 'ddos', 'dos', 'xss', 'csrf', 'sql', 'nosql',
    'rest', 'soap', 'grpc', 'graphql', 'crud', 'mvc', 'mvvm',
    'oop', 'fp', 'tdd', 'bdd', 'ci', 'cd', 'devops', 'sre',
    
    # === PROGRAMMING LANGUAGES & FRAMEWORKS ===
    'html', 'css', 'xml', 'json', 'yaml', 'sql', 'php', 'asp',
    'jsp', 'js', 'ts', 'jsx', 'tsx', 'vue', 'svelte',
    'ios', 'macos', 'tvos', 'watchos', 'ipados',
    'aws', 'gcp', 'azure', 'ibm', 'sap', 'erp', 'crm',
    
    # === FILE FORMATS & EXTENSIONS ===
    'pdf', 'doc', 'docx', 'xls', 'xlsx', 'ppt', 'pptx',
    'jpg', 'jpeg', 'png', 'gif', 'svg', 'bmp', 'tiff', 'webp',
    'mp3', 'mp4', 'avi', 'mkv', 'mov', 'wmv', 'flv', 'webm',
    'wav', 'flac', 'aac', 'ogg', 'm4a', 'wma',
    'zip', 'rar', 'tar', 'gz', '7z', 'iso', 'dmg',
    'exe', 'dll', 'bat', 'sh', 'app', 'apk', 'ipa',
    'csv', 'tsv', 'txt', 'rtf', 'md', 'log',
    
    # === COMPANIES - TECH ===
    'ibm', 'hp', 'dell', 'amd', 'intel', 'nvidia', 'asus',
    'msi', 'ati', 'arm', 'tsmc', 'qualcomm',
    'att', 'verizon', 'tmobile', 'sprint',
    'sap', 'oracle', 'cisco', 'vmware', 'redhat',
    
    # === COMPANIES - OTHER ===
    'ge', 'gm', 'ford', 'bmw', 'vw', 'kfc', 'mcdonalds',
    'ups', 'fedex', 'dhl', 'usps', 'tnt',
    'bp', 'shell', 'exxon', 'chevron', 'opec',
    
    # === MEDIA & ENTERTAINMENT ===
    'tv', 'dvd', 'cd', 'hd', 'uhd', '4k', '8k', 'hdr', 'sdr',
    'fm', 'am', 'dab', 'xm', 'sirius',
    'espn', 'hbo', 'mtv', 'vh1', 'bet', 'amc', 'tnt', 'tbs',
    'bbc', 'cnn', 'msnbc', 'fox', 'nbc', 'abc', 'cbs', 'pbs',
    'npr', 'pbs', 'hgtv', 'tlc', 'bravo', 'e!',
    'nfl', 'nba', 'mlb', 'nhl', 'mls', 'ufc', 'wwe', 'aew',
    'fifa', 'uefa', 'ioc', 'ncaa', 'pga', 'lpga',
    
    # === EDUCATION & DEGREES ===
    'phd', 'mba', 'md', 'jd', 'dds', 'dvm', 'pharmd',
    'bs', 'ba', 'ms', 'ma', 'mfa', 'mph', 'msw',
    'ged', 'sat', 'act', 'gre', 'gmat', 'lsat', 'mcat',
    'ap', 'ib', 'stem', 'steam',
    
    # === UNIVERSITIES ===
    'mit', 'ucla', 'usc', 'nyu', 'ucsb', 'ucsd', 'uci',
    'ucf', 'usf', 'uf', 'fsu', 'lsu', 'osu', 'psu',
    'byu', 'smu', 'tcu', 'uva', 'vcu', 'vmi', 'vt',
    
    # === BUSINESS & FINANCE ===
    'ceo', 'cfo', 'cto', 'coo', 'cmo', 'cio', 'cso', 'cdo',
    'vp', 'svp', 'evp', 'hr', 'pr', 'rd', 'qa', 'qc',
    'llc', 'inc', 'ltd', 'corp', 'plc', 'gmbh', 'sa',
    'ipo', 'roi', 'ebitda', 'kpi', 'roi', 'roa', 'roe',
    'gdp', 'gnp', 'cpi', 'ppi', 'gdp', 'imf', 'wto',
    'nyse', 'nasdaq', 'dow', 'sp', 'ftse', 'dax',
    'etf', 'ira', '401k', 'roth', 'hsa', 'fsa',
    'apr', 'apy', 'arm', 'fico', 'ach', 'eft', 'atm',
    
    # === AUTOMOTIVE ===
    'suv', 'mpv', 'rv', 'atv', 'utv', '4wd', 'awd', 'fwd', 'rwd',
    'mpg', 'mph', 'rpm', 'hp', 'bhp', 'psi', 'abs', 'esp',
    'ac', 'dc', 'ev', 'phev', 'hev', 'ice', 'cvt', 'dct',
    'vin', 'dmv', 'dui', 'dwi', 'drl', 'led', 'hid', 'oem',
    
    # === SCIENCE & MEASUREMENT ===
    'nasa', 'esa', 'jaxa', 'isro', 'iss', 'ufo', 'uap',
    'gps', 'gis', 'lidar', 'radar', 'sonar',
    'uv', 'ir', 'rf', 'em', 'led', 'lcd', 'oled', 'qled',
    'ac', 'dc', 'hz', 'khz', 'mhz', 'ghz', 'thz',
    'kb', 'mb', 'gb', 'tb', 'pb', 'kbps', 'mbps', 'gbps',
    
    # === SOCIAL MEDIA & INTERNET ===
    'seo', 'sem', 'ppc', 'cpc', 'cpm', 'ctr', 'roi',
    'url', 'uri', 'www', 'tld', 'ccTLD', 'gtld',
    'rss', 'xml', 'ajax', 'cors', 'jwt', 'oauth',
    'dm', 'pm', 'im', 'sms', 'mms', 'rcs',
    'gif', 'meme', 'nsfw', 'sfw', 'tldr', 'eli5',
    
    # === COMMON ABBREVIATIONS & SLANG ===
    'asap', 'rsvp', 'fyi', 'btw', 'omg', 'lol', 'lmao', 'rofl',
    'brb', 'afk', 'ttyl', 'gtg', 'idk', 'imo', 'imho',
    'tbh', 'smh', 'fomo', 'yolo', 'bae', 'bff', 'ily',
    'diy', 'faq', 'tba', 'tbd', 'eta', 'asap', 'rip',
    'vip', 'mvp', 'goat', 'aka', 'fka', 'dba',
    'etc', 'ie', 'eg', 'ps', 'pps', 'vs', 'aka',
    'am', 'pm', 'bc', 'ad', 'bce', 'ce',
    
    # === ORGANIZATIONS ===
    'un', 'nato', 'opec', 'asean', 'apec', 'brics',
    'eu', 'nafta', 'usmca', 'wto', 'imf', 'who',
    'unicef', 'unesco', 'interpol', 'icrc',
    'ngo', 'npo', 'ong', 'ingo',
    
    # === LEGAL & LAW ===
    'llc', 'inc', 'ltd', 'corp', 'pa', 'pc', 'lp', 'llp',
    'dba', 'aka', 'fka', 'nka', 'poa', 'nda', 'tos',
    'eula', 'gdpr', 'hipaa', 'ferpa', 'coppa', 'dmca',
    
    # === MISC ===
    'pos', 'pin', 'ssn', 'ein', 'tin', 'vin', 'isbn', 'issn',
    'upc', 'sku', 'ean', 'qr', 'rfid', 'nfc', 'ble',
    'pda', 'gps', 'pos', 'atm', 'pos', 'eft', 'ach',
    'hvac', 'led', 'lcd', 'crt', 'plasma', 'oled',
    'jpeg', 'mpeg', 'ascii', 'utf', 'unicode',
}

print(f"✓ Loaded {len(KNOWN_ACRONYMS)} known acronyms")
print("✓ Libraries ready!")

## Step 4: Define Cleanup Functions

In [ ]:
def should_remove_word(word):
    """
    Check if a word should be removed.
    
    Removes:
    - Words ending with apostrophe (word')
    - 2-letter lowercase words (except 'a' and 'i')
    """
    if pd.isna(word) or word == "":
        return True
    
    word_str = str(word).strip()
    
    # Remove words ending with apostrophe
    if word_str.endswith("'"):
        return True
    
    # Remove 2-letter lowercase words (except 'a' and 'i')
    if len(word_str) == 2 and word_str.islower():
        if word_str not in ['a', 'i']:
            return True
    
    return False

def fix_acronym(word):
    """
    Fix acronyms that should be fully uppercase.
    
    Examples:
    - Ny → NY
    - Fbi → FBI
    - Covid → COVID
    - Nyc → NYC
    - Usc → USC
    """
    if pd.isna(word) or word == "":
        return word
    
    word_str = str(word).strip()
    word_lower = word_str.lower()
    
    # Check if it's a known acronym
    if word_lower in KNOWN_ACRONYMS:
        return word_str.upper()
    
    # Keep original capitalization
    return word_str

print("✓ Cleanup functions defined!")
print("\nRules:")
print("  ✗ Words ending with ' (apostrophe)")
print("  ✗ 2-letter lowercase words (except 'a', 'i')")
print(f"  ✓ Fix {len(KNOWN_ACRONYMS)} known acronyms to uppercase")

## Step 5: Load CSV File

In [ ]:
# Load the CSV file
print("Loading valid_words.csv...")
df = pd.read_csv('valid_words.csv')

print(f"✓ Loaded {len(df):,} words")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst 10 rows:")
print(df.head(10))

## Step 6: Identify Words to Remove

In [ ]:
print("Analyzing words for removal...\n")

# Mark words for removal
df['should_remove'] = df['word'].apply(should_remove_word)

# Separate into keep and remove
to_remove = df[df['should_remove'] == True]
to_keep = df[df['should_remove'] == False]

print("=" * 60)
print("REMOVAL ANALYSIS")
print("=" * 60)
print(f"\nTotal words: {len(df):,}")
print(f"Words to keep: {len(to_keep):,}")
print(f"Words to remove: {len(to_remove):,}")

# Show examples of words to remove
if len(to_remove) > 0:
    print("\nExamples of words to remove:")
    
    # Words ending with apostrophe
    apostrophe_words = to_remove[to_remove['word'].str.endswith("'", na=False)]
    if len(apostrophe_words) > 0:
        print(f"\n  Ending with ' ({len(apostrophe_words):,} words):")
        print(f"    {apostrophe_words['word'].head(10).tolist()}")
    
    # 2-letter lowercase words
    two_letter = to_remove[to_remove['word'].apply(lambda x: len(str(x)) == 2 and str(x).islower())]
    if len(two_letter) > 0:
        print(f"\n  2-letter lowercase ({len(two_letter):,} words):")
        print(f"    {two_letter['word'].head(20).tolist()}")

## Step 7: Fix Acronyms

In [ ]:
print("\nFixing acronyms...\n")

# Apply acronym fixes to words we're keeping
to_keep['fixed_word'] = to_keep['word'].apply(fix_acronym)

# Find words that were changed
changed = to_keep[to_keep['word'] != to_keep['fixed_word']]

print("=" * 60)
print("ACRONYM FIXES")
print("=" * 60)
print(f"\nWords changed: {len(changed):,}")

if len(changed) > 0:
    print("\nExamples of acronym fixes:")
    print("\nOriginal → Fixed")
    print("-" * 30)
    for _, row in changed.head(30).iterrows():
        print(f"{row['word']:15s} → {row['fixed_word']}")

## Step 8: Create Cleaned Dataset

In [ ]:
print("\nCreating cleaned dataset...\n")

# Create final dataframe with fixed words
cleaned_df = to_keep[['fixed_word', 'count']].copy()
cleaned_df = cleaned_df.rename(columns={'fixed_word': 'word'})

# Deduplicate and merge counts
print("Checking for duplicates after acronym fixes...")
before_dedup = len(cleaned_df)
cleaned_df = cleaned_df.groupby('word', as_index=False).agg({'count': 'sum'})
after_dedup = len(cleaned_df)

print(f"  Before deduplication: {before_dedup:,}")
print(f"  After deduplication: {after_dedup:,}")
print(f"  Duplicates merged: {before_dedup - after_dedup:,}")

# Sort by count descending
cleaned_df = cleaned_df.sort_values('count', ascending=False)

print("\n" + "=" * 60)
print("FINAL STATISTICS")
print("=" * 60)
print(f"\nOriginal words: {len(df):,}")
print(f"Removed: {len(to_remove):,}")
print(f"Final cleaned words: {len(cleaned_df):,}")
print(f"\nReduction: {len(df) - len(cleaned_df):,} words ({((len(df) - len(cleaned_df)) / len(df) * 100):.2f}%)")

## Step 9: Preview Results

In [ ]:
print("\n" + "=" * 60)
print("PREVIEW OF CLEANED WORDS")
print("=" * 60)

# Show top 20 words
print("\nTop 20 most frequent words:")
print(cleaned_df.head(20).to_string(index=False))

# Show some acronyms if they exist
acronyms_in_list = cleaned_df[cleaned_df['word'].str.isupper() & (cleaned_df['word'].str.len() <= 6)]
if len(acronyms_in_list) > 0:
    print(f"\nSample acronyms in list ({len(acronyms_in_list):,} total):")
    print(acronyms_in_list.head(30).to_string(index=False))

## Step 10: Save Cleaned Results

In [ ]:
# Save cleaned data
print("\nSaving cleaned data...")

cleaned_df.to_csv('cleaned_words.csv', index=False)
print(f"✓ Saved {len(cleaned_df):,} words to 'cleaned_words.csv'")

# Also save the removed words for review
if len(to_remove) > 0:
    to_remove[['word', 'count']].to_csv('removed_words.csv', index=False)
    print(f"✓ Saved {len(to_remove):,} removed words to 'removed_words.csv' (for review)")

print("\n" + "=" * 60)
print("PROCESSING COMPLETE!")
print("=" * 60)
print("\nDownload files from the Files panel:")
print("  - cleaned_words.csv (final cleaned list)")
print("  - removed_words.csv (removed words for review)")

## Step 11: (Optional) Detailed Analysis

In [ ]:
print("\n" + "=" * 60)
print("DETAILED ANALYSIS")
print("=" * 60)

# Word length distribution
cleaned_df['length'] = cleaned_df['word'].apply(len)
print("\nWord length distribution:")
print(cleaned_df['length'].value_counts().sort_index().head(15))

# Capitalization breakdown
print("\nCapitalization breakdown:")
lowercase = len(cleaned_df[cleaned_df['word'].str.islower()])
uppercase = len(cleaned_df[cleaned_df['word'].str.isupper()])
capitalized = len(cleaned_df[cleaned_df['word'].str[0].str.isupper() & ~cleaned_df['word'].str.isupper()])

print(f"  Lowercase: {lowercase:,}")
print(f"  UPPERCASE (acronyms): {uppercase:,}")
print(f"  Capitalized: {capitalized:,}")

# Count statistics
print("\nCount statistics:")
print(f"  Highest: {cleaned_df['count'].max():,}")
print(f"  Lowest: {cleaned_df['count'].min():,}")
print(f"  Average: {cleaned_df['count'].mean():.0f}")
print(f"  Median: {cleaned_df['count'].median():.0f}")

## Summary

### Comprehensive Acronym Coverage (500+):

This notebook includes **500+ acronyms** across categories:

- **US States**: NY, CA, TX, FL, etc. (50)
- **Health**: HIV, AIDS, COVID, DNA, RNA, SARS, etc.
- **Technology**: API, CPU, GPU, RAM, HTML, CSS, SQL, etc.
- **Companies**: IBM, HP, AMD, FBI, CIA, NASA, etc.
- **Media**: ESPN, HBO, MTV, CNN, BBC, NFL, NBA, etc.
- **Education**: PhD, MBA, MIT, UCLA, USC, NYU, etc.
- **Business**: CEO, CFO, CTO, LLC, GDP, SEO, etc.
- **Internet**: LOL, OMG, IDK, BTW, FYI, ASAP, etc.
- **And many more...**

### Example Transformations:

```
word'     → REMOVED
ab        → REMOVED
Ny        → NY
Nyc       → NYC
Usc       → USC
Fbi       → FBI
Covid     → COVID
Hiv       → HIV
Dna       → DNA
Api       → API
Seo       → SEO
```